# selenium-yahoo-finance-scraper

Use the "Run" button to execute the code.

In [1]:
!pip install jovian --upgrade --quiet
!pip install selenium --upgrade --quiet

##prereq

- !pip install webdriver-manager --upgrade --quiet
- download required chromdriver and place it in the project path 

![](https://imgur.com/caHD7Dm.png)

![](https://imgur.com/H2LfYlh.png)


![](https://imgur.com/7jMFOcE.png)


![](https://media.giphy.com/media/TwLrWWtF6zcDGk5BQf/giphy.gif)

In [2]:
import jovian

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="selenium-yahoo-finance-scraper")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/selenium-yahoo-finance-scraper" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper


'https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper'

In [4]:
!pip install webdriver-manager --upgrade --quiet

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [6]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')
    serv = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=serv)
    driver = webdriver.Chrome(options=chrome_options, service=serv)
    driver.get(YAHOO_FINANCE_URL)
    return driver

In [7]:
def get_table_rows(driver):
    TABLE_CLASS = "W(100%)"  
    tablerows = len(driver.find_elements(By.XPATH, value="//table[@class= '{}']/tbody/tr".format(TABLE_CLASS)))
    return tablerows

In [8]:
def get_table_header(driver):
    header = driver.find_elements(By.TAG_NAME, value= 'th')
    header_list = [item.text for index, item in enumerate(header) if index < 10]
    return header_list

In [9]:
'''
rownum = 1
colnum = 3
driver.find_element(By.XPATH, value="//tr[{}]/td[{}]".format(rownum,colnum)).text
'''

'\nrownum = 1\ncolnum = 3\ndriver.find_element(By.XPATH, value="//tr[{}]/td[{}]".format(rownum,colnum)).text\n'

In [10]:
def parse_table_rows(rownum, driver, header_list):
    row_dictionary = {}
    for index , item in enumerate(header_list):
        row_dictionary[item] = driver.find_element(By.XPATH, value="//tr[{}]/td[{}]".format(rownum, index+1)).text
    return row_dictionary

In [11]:
YAHOO_FINANCE_URL = 'https://finance.yahoo.com/cryptocurrencies'#'https://finance.yahoo.com/trending-tickers'

print('Creating driver')
driver = get_driver()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758


Creating driver


Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/vinoddhole/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [12]:
header_list = get_table_header(driver)

In [13]:
table_rows = get_table_rows(driver)
table_rows

25

In [14]:
TOTAL_PAGES_TO_SCRAPE = 10
table_data = []
next_button_class = '//*[@id="scr-res-table"]/div[2]/button[3]'
for page in range(1, TOTAL_PAGES_TO_SCRAPE + 1):
    
    print('Getting Table row count for Page : {}'.format(page))
    table_rows = get_table_rows(driver)
    
    print('Parsing Page : {}'.format(page))
    table_data += [parse_table_rows(i, driver, header_list) for i in range (1, table_rows + 1)]
    
    print('Clicking Next Button')
    next_button = driver.find_elements(By.XPATH, value = next_button_class)    
    next_button[0].click()

Getting Table row count for Page : 1
Parsing Page : 1
Clicking Next Button
Getting Table row count for Page : 2
Parsing Page : 2
Clicking Next Button
Getting Table row count for Page : 3
Parsing Page : 3


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=98.0.4758.109)
Stacktrace:
0   chromedriver                        0x0000000102501ee9 chromedriver + 5013225
1   chromedriver                        0x000000010248d1d3 chromedriver + 4534739
2   chromedriver                        0x0000000102063a68 chromedriver + 170600
3   chromedriver                        0x00000001020668b1 chromedriver + 182449
4   chromedriver                        0x00000001020666d1 chromedriver + 181969
5   chromedriver                        0x000000010206696c chromedriver + 182636
6   chromedriver                        0x0000000102092b59 chromedriver + 363353
7   chromedriver                        0x00000001020b57e2 chromedriver + 505826
8   chromedriver                        0x000000010208cbf5 chromedriver + 338933
9   chromedriver                        0x00000001020b58ee chromedriver + 506094
10  chromedriver 

In [15]:
len(table_data)

50

In [16]:
print('Save the data to a CSV')
table_df = pd.DataFrame(table_data)
#print(table_df)
table_df.to_csv('cryptocurrencies.csv', index=None)
#### add timer 

Save the data to a CSV


In [17]:
table_df

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,"43,997.25",+551.32,+1.27%,834.729B,33.35B,33.35B,33.35B,18.972M
1,ETH-USD,Ethereum USD,"2,986.01",+64.22,+2.20%,357.698B,19.552B,19.552B,19.552B,119.791M
2,USDT-USD,Tether USD,1.0004,+0.0002,+0.02%,79.476B,77.682B,77.682B,77.682B,79.442B
3,BNB-USD,Binance Coin USD,411.93,-3.66,-0.88%,68.016B,2.729B,2.729B,2.729B,165.117M
4,USDC-USD,USD Coin USD,0.999920,+0.000222,+0.02%,53.613B,5.706B,5.706B,5.706B,53.617B
5,XRP-USD,XRP USD,0.768539,-0.003537,-0.46%,36.851B,2.94B,2.94B,2.94B,47.949B
6,LUNA1-USD,Terra USD,93.63,+4.86,+5.47%,35.078B,5.35B,5.35B,5.35B,374.653M
7,SOL-USD,Solana USD,103.68,+6.89,+7.12%,33.241B,3.779B,3.779B,3.779B,320.598M
8,ADA-USD,Cardano USD,0.957962,-0.010902,-1.13%,32.246B,1.709B,1.709B,1.709B,33.661B
9,HEX-USD,HEX USD,0.133675,-0.008138,-5.74%,23.181B,28.122M,28.122M,28.122M,173.411B


In [18]:
jovian.commit(project="selenium-yahoo-finance-scraper",git_commit=True)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/selenium-yahoo-finance-scraper" on https://jovian.ai/
[main e5ac23c] jovian commit vinodvidhole/selenium-yahoo-finance-scraper v52
 6 files changed, 1165 insertions(+), 894 deletions(-)
 rewrite cryptocurrencies.csv (99%)
 rewrite stock-market-news.csv (68%)
 delete mode 100644 trending-tickers.csv
/Users/vinoddhole/desktop/vinod/datascience/webScraping/yahoo-finance-scraper
[jovian] Git repository identified. Performing git commit...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper


'https://jovian.ai/vinodvidhole/selenium-yahoo-finance-scraper'